In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
import pandas as pd

In [ ]:
## Buoc 1: Import anh

data = './dogs-vs-cats'
train_data = data + '/train'
test_data = data + '/test1'

In [ ]:
def get_img(folder):
    # images = {"features": None, "labels": None}
    features = []
    labels = []
    for filename in os.listdir(folder):
        # img = cv2.imread(os.path.join(folder, filename))
        # resized = cv2.resize(img, (224, 224))
        # # resized = img_to_array(resized)
        # if img is not None:
        #     features.append(resized)
        if "dog" in filename:
            labels.append("dog")
        else:
            labels.append("cat")
    # images["features"] = features
    # images["labels"] = labels
    df = pd.DataFrame({
        "features": filename,
        "labels": labels
    })
    return df

In [ ]:
train_images = get_img(train_data)



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, valid = train_test_split(train_images, test_size=0.3, random_state=42)
train = train.reset_index(drop=True)
valid = train.reset_index(drop=True)


In [ ]:
# Buoc 2: tien xu ly anh
train_datagen = ImageDataGenerator(rotation_range=0.20, rescale=1/255, height_shift_range=0.2, zoom_range=0.5, horizontal_flip=True,
                                   vertical_flip=True)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train,
    train_data,
    x_col='features',
    y_col='labels',
    target_size=(224,224),
    class_mode='binary',
    batch_size=64,
    color_mode='rgb',
    shuffle=True
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_dataframe(
    valid,
    train_data,
    x_col='features',
    y_col='labels',
    target_size=(224,224),
    class_mode='binary',
    batch_size=64,
    color_mode='rgb',
    shuffle=True
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, \
    MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD

In [ ]:
# Buoc 3: Xay dung model VGG16

In [ ]:
width = 224
height = 224
classes = 2
shape = (width, height, 3)

In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=shape),
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128, (3, 3), activation="relu", padding="same"),
    Conv2D(128, (3, 3), activation="relu", padding="same"),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(256, (3, 3), activation="relu", padding="same"),
    Conv2D(256, (3, 3), activation="relu", padding="same"),
    Conv2D(256, (3, 3), activation="relu", padding="same"),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(1000, activation='relu'),
    Dense(1, activation='softmax'),
])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9), loss="binary_crossentropy", metrics= ["accuracy"] )

In [ ]:
epochs = 10
batch_size = 64

In [ ]:
print("starting training ...")

history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train)/64,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(valid)/64
)